# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [2]:
# Your code here
import requests
from bs4 import BeautifulSoup

base_url = 'http://books.toscrape.com/'
response = requests.get(base_url)

print(response.status_code)

200


# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [3]:
# Your code here
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

category_links = soup.select('.nav-list a')

category_urls = [link.get('href') for link in category_links]
print(category_urls)

['catalogue/category/books_1/index.html', 'catalogue/category/books/travel_2/index.html', 'catalogue/category/books/mystery_3/index.html', 'catalogue/category/books/historical-fiction_4/index.html', 'catalogue/category/books/sequential-art_5/index.html', 'catalogue/category/books/classics_6/index.html', 'catalogue/category/books/philosophy_7/index.html', 'catalogue/category/books/romance_8/index.html', 'catalogue/category/books/womens-fiction_9/index.html', 'catalogue/category/books/fiction_10/index.html', 'catalogue/category/books/childrens_11/index.html', 'catalogue/category/books/religion_12/index.html', 'catalogue/category/books/nonfiction_13/index.html', 'catalogue/category/books/music_14/index.html', 'catalogue/category/books/default_15/index.html', 'catalogue/category/books/science-fiction_16/index.html', 'catalogue/category/books/sports-and-games_17/index.html', 'catalogue/category/books/add-a-comment_18/index.html', 'catalogue/category/books/fantasy_19/index.html', 'catalogue/

# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [4]:
# Your code here
def get_book_urls(category_url):
    response = requests.get(base_url + category_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    book_links = soup.select('h3 a')
    book_urls = [base_url + 'catalogue/' + link.get('href') for link in book_links]
    return book_urls

example_category_url = 'catalogue/category/books/travel_2/index.html'
book_urls = get_book_urls(example_category_url)
print(book_urls)

['http://books.toscrape.com/catalogue/../../../its-only-the-himalayas_981/index.html', 'http://books.toscrape.com/catalogue/../../../full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html', 'http://books.toscrape.com/catalogue/../../../see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html', 'http://books.toscrape.com/catalogue/../../../vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html', 'http://books.toscrape.com/catalogue/../../../under-the-tuscan-sun_504/index.html', 'http://books.toscrape.com/catalogue/../../../a-summer-in-europe_458/index.html', 'http://books.toscrape.com/catalogue/../../../the-great-railway-bazaar_446/index.html', 'http://books.toscrape.com/catalogue/../../../a-year-in-provence-provence-1_421/index.html', 'http://books.toscrape.com/catalogue/../../../the-road-to-little-dribbling-adventures-of-an-american-in-britain-notes-from-a-small-island-2_277/index.html', 'http://books.toscrape.com

# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [5]:
# Your code here
def get_book_details(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    title = soup.h1.text
    price = soup.select_one('.price_color').text
    availability = soup.select_one('.instock.availability').text.strip()
    rating = len(soup.select('p.star-rating')[0].get('class')[1])
    description = soup.select_one('#product_description + p').text if soup.select_one('#product_description + p') else 'No description available'
    upc = soup.select_one('th:contains("UPC") + td').text

    return {
        "Title": title,
        "Price": price,
        "Availability": availability,
        "Rating": rating,
        "Description": description,
        "UPC": upc
    }

example_book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
book_details = get_book_details(example_book_url)
print(book_details)

{'Title': 'A Light in the Attic', 'Price': 'Â£51.77', 'Availability': 'In stock (22 available)', 'Rating': 5, 'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to 

/opt/anaconda3/lib/python3.12/site-packages/soupsieve/css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [6]:
import pandas as pd
from tqdm import tqdm

books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": []}

for category_url in tqdm(category_urls):
    book_urls = get_book_urls(category_url)
    for book_url in book_urls:
        details = get_book_details(book_url)
        details['Category'] = category_url.split('/')[-2]  
        for key in books_dict.keys():
            books_dict[key].append(details[key])
            
books_df = pd.DataFrame(books_dict)
print(books_df.head())